# Character Extractor from CSV and Image
### v2.0.5

In [1]:
import os
import csv
import cv2
import numpy as np
from itertools import groupby
from tqdm.notebook import tqdm

class ExtractCharacter:
    def __init__(self):
        #type_data = "test"
        self.__img_path = "../../../sources/4-release/80-20/"+type_data+"/images"
        self.__f_csv = "../../../sources/4-release/80-20/"+type_data+"/"+type_data+".csv"
        path_save = type_data
        
        self.__char_path = os.path.join(path_save, "characters")
        self.__plate_path = os.path.join(path_save, "plates")
        self.__path_group = [path_save,
                             self.__char_path,
                             self.__plate_path
                           ]
        
    def checking_path(self):
        for path in self.__path_group:
            if not(os.path.exists(path)):
                os.mkdir(path)
        print('path checked')
        
    def extracting_location(self, location):
        w1, h1, w2, h2 = map(int, location.split('_'))
        he1 = h1
        he2 = h1+h2
        wi1 = w1
        wi2 = w1+w2
        plate_coordinates = [he1, he2, wi1, wi2] # Plate coordinate
        
        return plate_coordinates
    
    def loading_method(self, csv_file):
        # csv = [file_name.jpg, 'plat_number', 'char location 1', 'char location 2', cont...]
        csv_data = []
        with open(csv_file) as f:
            reader = csv.reader(f)
            next(reader, None)
            
            for row in reader:
                image_name = row[0]
                plate_location = image_name.split('.')[0].split('-')[1]
                extracted_location = self.extracting_location(plate_location)
                
                plate_number = row[1].replace('_', '')
                char_number = len(plate_number)
                char_list = list(plate_number)
                
                char_group = []
                up = 0
                for char_coordinate in range(2, char_number+2):
                    w, h, w1, h1 = map(int, row[char_coordinate].replace('[','').replace(']','').split(','))
                    he1 = h
                    he2 = h + h1
                    wi1 = w
                    wi2 = w + w1
                    char = char_list[up]
                    char_group.append([char, he1, he2, wi1, wi2])
                    up += 1
                csv_data.append([image_name, extracted_location, plate_number, char_group])
        
        return csv_data
        
    def load_csv(self):
        self.__csv_data = self.loading_method(self.__f_csv)
        
    def loading_character(self):
        plate_path = self.__plate_path
        char_path = self.__char_path
        csv_data = self.__csv_data
        
        # Fname, plate_coordinates, plate_number, characters
        name = 1
        for row in tqdm(csv_data):
            img = cv2.imread(os.path.join(self.__img_path, row[0]), cv2.IMREAD_GRAYSCALE)
            cropped_plate = img[row[1][0]:row[1][1], row[1][2]:row[1][3]]
            cv2.imwrite(os.path.join(plate_path, row[0]), cropped_plate) # Save Plates
            
            for char, height1, height2, width1, width2 in row[3]:
                cropped_img = img[height1:height2, width1:width2]
                
                path = os.path.join(char_path, char)
                if not os.path.exists(path):
                    os.mkdir(path)
                cv2.imwrite(os.path.join(path, str(name)+".jpg"), cropped_img)
                name += 1

if __name__=='__main__':
    ec = ExtractCharacter()
    ec.checking_path()
    ec.load_csv()
    ec.loading_character()

path checked
